<a href="https://colab.research.google.com/github/kbrezinski/Applied-ML-for-Scientists/blob/main/test_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# Load the Deepseek Coder model and its corresponding tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
# Use the same model_id for the tokenizer to ensure compatibility
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Set the padding token (using the eos_token from the model's tokenizer)
tokenizer.pad_token = tokenizer.eos_token

print(f"Model '{model_id}' and its corresponding tokenizer loaded successfully.")
print(f"Padding token set to: {tokenizer.pad_token}")

# Quick test of the tokenizer
toy_text = "def hello_world():\n  print('Hello, world!')"
encoded_input = tokenizer(toy_text, return_tensors="pt")

print("\nToy Text:")
print(toy_text)
print("\nEncoded Input (input_ids):")
print(encoded_input.input_ids)
print("\nDecoded Output:")
print(tokenizer.decode(encoded_input.input_ids[0]))

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_input.input_ids[0])
print("\nRaw Tokens:")
print(tokens)

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
# Define the tokenization function
def tokenize_function(examples):
    # Concatenate the instruction and output fields with a separator
    # Adjust this based on the specific dataset format and desired fine-tuning task
    text = [f"Instruction: {inst}\nOutput: {out}" for inst, out in zip(examples["instruction"], examples["output"])]
    return tokenizer(text, truncation=True, padding="max_length", max_length=512) # Adjust max_length as needed

# Apply the tokenization function to the dataset
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

print("Dataset tokenized successfully.")
print(tokenized_dataset)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

Dataset tokenized successfully.
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 18612
    })
})


In [ ]:
# Display a single row from the tokenized dataset
print("Displaying a single row from the tokenized dataset:")
from IPython.display import display
display(tokenized_dataset["train"][0])

In [ ]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-model",  # Output directory for checkpoints and logs
    per_device_train_batch_size=4, # Adjust based on your GPU memory
    gradient_accumulation_steps=2, # Adjust based on your GPU memory and batch size
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    eval_strategy="no", # Set evaluation strategy to "no" since there's no evaluation dataset
    eval_steps=500, # Evaluate every 500 steps (this setting will be ignored with eval_strategy="no")
    load_best_model_at_end=False, # No evaluation, so no best model to load
    metric_for_best_model="eval_loss", # This metric will be ignored with eval_strategy="no"
    greater_is_better=False, # This will be ignored with eval_strategy="no"
    report_to="none" # Or "tensorboard", "wandb", etc.
)

print("Training arguments configured.")
print(training_args)

Training arguments configured.
TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=500,
eval_strategy=IntervalStrategy.NO,


In [ ]:
from transformers import Trainer

# Set up the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_dataset["train"], # training dataset
    # eval_dataset=tokenized_dataset["validation"], # evaluation dataset (optional)
    processing_class=tokenizer,                 # the tokenizer used to preprocess the data
)

print("Trainer set up successfully.")

NameError: name 'model' is not defined

In [ ]:
# Start training
trainer.train()

print("Training complete!")

NameError: name 'trainer' is not defined